In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime 
%matplotlib inline

train_df = pd.read_csv("train.csv", parse_dates=['timestamp'])
test_df = pd.read_csv("test.csv")

Quantify categorical columns

In [2]:
string_dict = {'no data':-1, 
              'poor':0,
              'satisfactory':1,
              'good': 2,
              'excellent':3,
              'yes': 1,
              'no': 0,
              'Investment': 1,
              'OwnerOccupier': 0}
for key in [key for key in dict(train_df.dtypes) if dict(train_df.dtypes)[key] in ['O'] and key != 'sub_area']:
    train_df[key] = train_df[key].map(string_dict)

Add one column for each sub_area

In [3]:
for area in train_df['sub_area'].unique():
    train_df.insert(train_df.shape[1],'sub_area_'+area, 0)
    train_df['sub_area_'+ area] = (train_df['sub_area'] == area).astype(int)
    
train_df = train_df.drop('sub_area', 1)

Edit 'build_year'

In [4]:
train_df.ix[train_df['build_year']==0,'build_year'] = None
train_df['build_year'] = train_df['build_year'].apply(lambda x: 2017-x)

Edit'timestamp' as days to date(2016,6,1)

In [5]:
train_df['timestamp'] = train_df['timestamp'].apply(lambda x: (datetime.datetime(2016,6,1)-x).days)
train_df['price_doc'] = train_df['price_doc'].apply(lambda x: np.log(x))

Normalization

In [6]:
train_df_dscb = train_df.describe(percentiles=[.96])

for col in [x for x in train_df_dscb.columns.values if x not in ['id'] and x[0:9] != 'sub_area_']:
    print(col, end='//')
    train_df[col] = train_df.ix[:, col].apply(lambda x: (x-train_df_dscb[col]['min'])/(train_df_dscb[col]['96%']-train_df_dscb[col]['min']))

timestamp//full_sq//life_sq//floor//max_floor//material//build_year//num_room//kitch_sq//state//product_type//area_m//raion_popul//green_zone_part//indust_part//children_preschool//preschool_quota//preschool_education_centers_raion//children_school//school_quota//school_education_centers_raion//school_education_centers_top_20_raion//hospital_beds_raion//healthcare_centers_raion//university_top_20_raion//sport_objects_raion//additional_education_raion//culture_objects_top_25//culture_objects_top_25_raion//shopping_centers_raion//office_raion//thermal_power_plant_raion//incineration_raion//oil_chemistry_raion//radiation_raion//railroad_terminal_raion//big_market_raion//nuclear_reactor_raion//detention_facility_raion//full_all//male_f//female_f//young_all//young_male//young_female//work_all//work_male//work_female//ekder_all//ekder_male//ekder_female//0_6_all//0_6_male//0_6_female//7_14_all//7_14_male//7_14_female//0_17_all//0_17_male//0_17_female//16_29_all//16_29_male//16_29_female//0_1

Fill the null cells with -1

In [7]:
train_df = train_df.fillna(-1)

Save processed data

In [8]:
train_df.to_csv("train_df_after_processing.csv", index=False)
train_df_dscb.to_csv("train_df_Normalization_bounds.csv", index=False)

In [9]:
y_train = np.array(train_df.ix[:,'price_doc'])
train_df_dscb['price_doc']

count    30471.000000
mean        15.609483
std          0.604575
min         11.512925
50%         15.651990
96%         16.556351
max         18.526041
Name: price_doc, dtype: float64

In [11]:
train_df['price_doc'].describe()

count    30471.000000
mean         0.812257
std          0.119874
min          0.000000
25%          0.765080
50%          0.820685
75%          0.876159
max          1.390546
Name: price_doc, dtype: float64